In [1]:
from numpy import linalg as LA
from scipy.sparse.linalg.eigen.arpack import eigsh as largest_eigsh
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics.cluster import adjusted_rand_score
from multi import *
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# MPPM

## Data Generation

In [2]:
## Parameters
n = 600
k = 2
L = 2
rho = 2*np.log(n)/n
pi = [1/2,1/2]
in_list = [4,4]
out_list = [2,3.5]

## Generate multi-layer adjacency matrices
Ares,gt = genPPM(n,k,L,rho,pi,in_list,out_list)

## Optimal weight
print(lam_norm(in_list,out_list,k=2))

[0.8333333333333334, 0.16666666666666669]


## Algorithms

SCME_gm

In [3]:
test = SCME(Ares,k,method = 'gmm')
label,lamb,eratio = test.optimize(itr = 10)
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.675 0.325]
ARI:  0.8647


SCME_km

In [4]:
test = SCME(Ares,k,method = 'km')
label,lamb,eratio = test.optimize(itr = 10)
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

Weights:  [0.7829 0.2171]
ARI:  0.9087


ISC_gm

In [ ]:
test = ISC(Ares,k,method = 'gmm')
lamb,label = test.opt()
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

ISC_km

In [ ]:
test = ISC(Ares,k,method = 'km')
lamb,label = test.opt()
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

ASC

In [ ]:
kmeans = ASC(Ares,k)
print(adjusted_rand_score(kmeans.labels_,gt))

MAM

In [ ]:
asso = [True]*L
test = MAM(Ares,asso,k)
_,mam = test.Mam()
print(adjusted_rand_score(SC(mam,k),gt))

Mean adj

In [ ]:
print(adjusted_rand_score(SC(sum([a*b for a,b in zip([1/L]*L,Ares)]),k),gt))

# SBM

## Data Generation

In [ ]:
## Parameters
n = 600
k = 2
L = 2
rho = 1.6*np.log(n)/n
pi = [1/2,1/2]
Alist = [np.array([[5,2],[2,4]]),np.array([[4,3.5],[3.5,5]])]

## Generate multi-layer adjacency matrices
Ares,gt = genSBM(n,k,L,rho,pi,Alist)

## Algorithms

SCME_gm

In [ ]:
test = SCME(Ares,k,method = 'gmm')
label,lamb,eratio = test.optimize(itr = 10)
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

SCME_km

In [ ]:
test = SCME(Ares,k,method = 'km')
label,lamb,eratio = test.optimize(itr = 10)
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

ISC_gm

In [ ]:
test = ISC(Ares,k,method = 'gmm')
lamb,label = test.opt()
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

ISC_km

In [ ]:
test = ISC(Ares,k,method = 'km')
lamb,label = test.opt()
print('Weights: ',np.round(lamb,4))
print('ARI: ',round(adjusted_rand_score(label,gt),4))

ASC

In [ ]:
kmeans = ASC(Ares,k)
print(adjusted_rand_score(kmeans.labels_,gt))

MAM

In [ ]:
asso = [True]*L
test = MAM(Ares,asso,k)
_,mam = test.Mam()
print(adjusted_rand_score(SC(mam,k),gt))

Mean adj

In [ ]:
evals, evecs = largest_eigsh(sum([a*b for a,b in zip([1/L]*L,Ares)]),k+1, which='LM')
evals_abs = abs(evals)
evecs = evecs[:,evals_abs.argsort()]
evals = evals[evals_abs.argsort()]
model = GaussianMixture(n_components=k,n_init=10).fit(np.real(evecs[:,1:]))
label = model.predict(np.real(evecs[:,1:]))
print(adjusted_rand_score(label,gt))